In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"
# mpl.rcParams["lines.markersize"] = 20

## Explore Link 11

In [ ]:
# Load traffic data + Incident (long format)
df = pd.read_csv('data/clean/df11ds.csv', index_col=0)
df.DateTime = pd.to_datetime(df.DateTime, format="%Y-%m-%d %H:%M:%S")
# Set DateTime as index and sort
df = df.set_index('DateTime')
df.index = pd.to_datetime(df.index)
if not df.index.is_monotonic:
    df = df.sort_index()
    
df.info()

In [ ]:
inc11 = pd.read_csv('data/clean/inc11_wide.csv', index_col=0)
inc12 = pd.read_csv('data/clean/inc12_wide.csv', index_col=0)

inc11.WST_Start = pd.to_datetime(inc11.WST_Start)
inc11.WST_End = pd.to_datetime(inc11.WST_End)
inc12.WST_Start = pd.to_datetime(inc12.WST_Start)
inc12.WST_End = pd.to_datetime(inc12.WST_End)

inc11.info()

In [ ]:
# inc11.Incident_Type = inc11.Incident_Type.replace([
#     'Flooding /\nStorm', 'Special Event /\nPothole / Hazmat',
#     'Special Event /\nPothole / Hazmat', 'Animal /\n Livestock',
#     'Vehicle Fire'], 'Miscellaneous')

In [ ]:
df.iloc[:,1:] \
    .describe() \
    .apply(lambda s: s.apply('{0:.4f}'.format))

In [ ]:
df.corr()['Volume']

In [ ]:
df.corr()['Speed']

In [ ]:
df.corr()['Occupancy']

In [ ]:
pd.plotting.scatter_matrix(df.iloc[:,1:4], alpha=0.05)
plt.show()

In [ ]:
sns.pairplot(df, vars=['Volume','Speed','Occupancy'],
             kind='scatter', diag_kind="kde",
             plot_kws={'color':'b', 'edgecolor':'face'})
plt.savefig('fig/Pair_diagram/L11.png')
plt.show()

In [ ]:
df2 = pd.read_csv('data/clean/LAD.csv')
df2['Density'] = df2['Occupancy']*4000/700
df2.info()

In [ ]:
def fund_plot(df, link):
    plt.rcParams["lines.markersize"] = 12
    temp = df[df.ID==link]
    v = temp.Volume.max()
    sv = round(temp[temp.Volume==v].iloc[0]['Speed'],1)
    dv = round(temp[temp.Volume==v].iloc[0]['Density'],1)
    d = temp.Density.max()
    sd = round(temp[temp.Density==d].iloc[0]['Speed'],1)
    s = temp.Speed.max()
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18,4))
    ax[0].scatter(temp.Volume, temp.Speed, alpha=0.5)
    ax[0].set_xlabel('Volume (veh/min)')
    ax[0].set_ylabel('Speed (km/hr)')  
    ax[0].annotate(f'Max Volume = {v}\nwith Speed={sv}',
                   xy=(v, sv), xytext=(v, s-50), ha='right',
                   arrowprops=dict(arrowstyle="->"))
    ax[1].scatter(temp.Density, temp.Volume, alpha=0.5)
    ax[1].set_xlabel('Density (veh/km)')
    ax[1].set_ylabel('Volume (veh/min)')
    ax[1].annotate(f'Max Volume = {v}\nwith Density={dv}',
                   xy=(dv, v), xytext=(dv+20, v-10), ha='left',
                   arrowprops=dict(arrowstyle="->"))
    ax[2].scatter(temp.Density, temp.Speed, alpha=0.15)
    ax[2].set_xlabel('Density (veh/km)')
    ax[2].set_ylabel('Speed (km/hr)')
    ax[2].annotate(f'Max Density = {round(d,1)}',
                   xy=(d, sd), xytext=(d, 100), ha='right',
                   arrowprops=dict(arrowstyle="->"))
    plt.savefig(f'fig/Pair_diagram/L{link}.png', bbox_inches='tight')
    plt.show()

for i in range(1,15):
    fund_plot(df2, i)

In [ ]:
df.iloc[:,1:4].plot.box(figsize=(6,6))
plt.show()

In [ ]:
# list of days that have incident(s)
date_ls = inc11['WST_Start'].dt.date.unique()

In [ ]:
def day_plot(df, idx):
    plt.rcParams["lines.markersize"] = 12
    plt.rcParams["font.size"] = 14
    fig, ax = plt.subplots(figsize=(10,4))
    ax.set_title(f'{date_ls[idx].strftime("%d %B %Y")}',
                 loc='right', fontsize=14)
    ax.set_ylim(-10,120)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    # Base plot (no incidents)
    temp = df[(df.index.date == date_ls[idx])]
    ax.plot(temp.Volume, color='k', zorder=1)
    ax.vlines(x=pd.Timestamp('2018-08-27 08:30:00'),
             ymin=-10, ymax=120, color='darkgrey', linestyle='--',
             label='Investigated\n period', linewidth=2)
    ax.vlines(x=pd.Timestamp('2018-08-27 13:30:00'),
             ymin=-10, ymax=120, color='darkgrey', linestyle='--', linewidth=2)
    
    # Main section incident 1
    temp1 = df[(df.index.date == date_ls[idx]) &
              (df.Incident_Type1 != 'None1')]
    if temp1.shape[0] != 0:
        temp1 = temp1.reindex(pd.date_range(temp1.index.min(),
                                    temp1.index.max(),
                                    freq='1min'), fill_value=np.nan)
        m1 = (temp1.loc[temp1['Incident_Type1'].shift(1) != temp1['Incident_Type1']]).dropna()
        ax.plot(temp1.Volume, color='r',  zorder=1)
        ax.scatter(m1.index, m1.Volume, color='red', marker="*", zorder=2)
        ax.annotate('Road Crash\n(Study Road)', 
               xy=(m1.index[0],m1.Volume[0]),
               xytext=(pd.to_datetime('2018-08-27 07:45:00'),10), ha='center',
               arrowprops=dict(color='darkred', arrowstyle="->", lw=2))
        ax.annotate('Animal /\nLivestock\n(Study Road)', 
               xy=(m1.index[1],m1.Volume[1]),
               xytext=(pd.to_datetime('2018-08-27 14:00:00'),10), ha='center',
               arrowprops=dict(color='darkred', arrowstyle="->", lw=2))
        
    # Main section incident 2
    temp2 = df[(df.index.date == date_ls[idx]) &
              (df.Incident_Type2 != 'None2')]
    if temp2.shape[0] != 0:
        temp2 = temp2.reindex(pd.date_range(temp2.index.min(),
                                    temp2.index.max(),
                                    freq='1min'), fill_value=np.nan)
        m2 = (temp2.loc[temp2['Incident_Type2'].shift(1) != temp2['Incident_Type2']]).dropna()
        ax.plot(temp2.Volume+2, color='r', zorder=1)
        ax.scatter(m2.index, m2.Volume, color='red', marker="*", zorder=2)
        
    # Downstream incident
    temp3 = df[(df.index.date == date_ls[idx]) &
              (df.Downstream_Incident == 1)]
    if temp3.shape[0] != 0:
        temp3 = temp3.reindex(pd.date_range(temp3.index.min(),
                                    temp3.index.max(),
                                    freq='1min'),fill_value=np.nan)
        d = (temp3.loc[temp3['Downstream_Incident'].shift(1) != temp3['Downstream_Incident']]).dropna()
        ax.plot(temp3.Volume, color='b', zorder=1)
        ax.scatter(d.index, d.Volume, color='blue', marker="*", zorder=2)
        ax.annotate('Road Crash\n(Downstream)', 
               xy=(d.index[0],d.Volume[0]),
               xytext=(pd.to_datetime('2018-08-27 07:45:00'),100), ha='center',
               arrowprops=dict(color='navy', arrowstyle="->", lw=2))
        ax.annotate('Break Down /\nTow Away\n(Downstream)',
               xy=(d.index[1],d.Volume[1]),
               xytext=(pd.to_datetime('2018-08-27 14:00:00'),90), ha='center',
               arrowprops=dict(color='navy', arrowstyle="->", lw=2))
        
    ax.legend(fontsize=12)
    plt.savefig('fig/L11/27Aug_volume.png', bbox_inches='tight')
    plt.show()

day_plot(df, 41)

In [ ]:
def period_plot(df, idx, start_dt, end_dt):
    plt.rcParams["lines.markersize"] = 12
    plt.rcParams["font.size"] = 14
    fig, ax = plt.subplots(figsize=(10,4))
    ax.set_title(f'{date_ls[idx].strftime("%d %B %Y")}',
                 loc='right', fontsize=14)
    ax.set_ylim(-10,120)
    ax.set_xticks(pd.date_range('2018-08-27 08:30:00', '2018-08-27 13:30:00', freq='1H'),
                  ['08:30', '09:30', '10:30', '11:30', '12:30', '13:30'])
    
    # Base plot (no incidents)
    temp = df[(df.index.date == date_ls[idx]) &
              (df.index >= start_dt) & (df.index < end_dt)]
    ax.plot(temp.Volume, color='k', zorder=1)
    ax.grid()
    
    # Main section incident 1
    temp1 = df[(df.index.date == date_ls[idx]) &
              (df.Incident_Type1 != 'None1')]
    if temp1.shape[0] != 0:
        temp1 = temp1.reindex(pd.date_range(temp1.index.min(),
                                    temp1.index.max(),
                                    freq='1min'), fill_value=np.nan)
        m1 = (temp1.loc[temp1['Incident_Type1'].shift(1) != temp1['Incident_Type1']]).dropna()
        ax.plot(temp1.Volume, color='r',  zorder=1)
        ax.scatter(m1.index, m1.Volume, color='red', marker="*", zorder=2)
        ax.annotate('Road Crash\n(Study Road)', 
               xy=(m1.index[0],m1.Volume[0]),
               xytext=(m1.index[0],10), ha='center',
               arrowprops=dict(color='darkred', arrowstyle="-|>", lw=2))
        ax.annotate('Animal /\nLivestock\n(Study Road)', 
               xy=(m1.index[1],m1.Volume[1]),
               xytext=(m1.index[1],80), ha='center',
               arrowprops=dict(color='darkred', arrowstyle="-|>", lw=2))
        
    # Main section incident 2
    temp2 = df[(df.index.date == date_ls[idx]) &
              (df.Incident_Type2 != 'None2')]
    if temp2.shape[0] != 0:
        temp2 = temp2.reindex(pd.date_range(temp2.index.min(),
                                    temp2.index.max(),
                                    freq='1min'), fill_value=np.nan)
        m2 = (temp2.loc[temp2['Incident_Type2'].shift(1) != temp2['Incident_Type2']]).dropna()
        ax.plot(temp2.Volume+2, color='r', zorder=1)
        ax.scatter(m2.index, m2.Volume, color='red', marker="*", zorder=2)
        
    # Downstream incident
    temp3 = df[(df.index.date == date_ls[idx]) &
              (df.Downstream_Incident == 1)]
    if temp3.shape[0] != 0:
        temp3 = temp3.reindex(pd.date_range(temp3.index.min(),
                                    temp3.index.max(),
                                    freq='1min'),fill_value=np.nan)
        d = (temp3.loc[temp3['Downstream_Incident'].shift(1) != temp3['Downstream_Incident']]).dropna()
        ax.plot(temp3.Volume, color='b', zorder=1)
        ax.scatter(d.index, d.Volume, color='blue', marker="*", zorder=2)
        ax.annotate('Road Crash\n(Downstream)', 
               xy=(d.index[0],d.Volume[0]),
               xytext=(d.index[0],10), ha='center',
               arrowprops=dict(color='navy', arrowstyle="-|>", lw=2))
        ax.annotate('Break Down /\nTow Away\n(Downstream)',
               xy=(d.index[1],d.Volume[1]),
               xytext=(d.index[1],80), ha='center',
               arrowprops=dict(color='navy', arrowstyle="-|>", lw=2))
    
    plt.savefig('fig/L11/27Aug_volume_2.png', bbox_inches='tight')
    plt.show()

period_plot(df, 41, '2018-08-27 08:30:00', '2018-08-27 13:30:00')